# TPC3: Processador de Pessoas listadas nos Róis de Confessados

Construa agora um ou vários programas Python para processar o texto 'processos.txt' (procurar o ficheiro no Bb) com o intuito de calcular frequências de alguns elementos (a ideia é utilizar arrays associativos, dicionáriosem Python, para o efeito) conforme solicitado a seguir:

### Bibliotecas importadas

In [ ]:
import re
import json
from prettytable import PrettyTable

### Funções auxiliares para impressão de distribuições

Alíneas a) e c)

In [ ]:
def print_distribuicao(table_title, table_fields, data):
    table = PrettyTable()
    table.title = table_title
    table.field_names = table_fields

    total = sum(data.values())
    for key in data:
        percentage = data[key] / total * 100
        
        precision = 2
        while round(percentage, precision) == 0.00:
            precision += 1

        table.add_row([key, data[key], '{:.{}f} %'.format(percentage, precision)])

    print(table)

Alínea b)

In [ ]:
def print_top5_nomes_por_seculo(table_title, table_fields, data):
    for seculo in data:
        if len(data[seculo]) > 0:    
            table = PrettyTable()
            table.title = table_title + " " + str(seculo)
            table.field_names = table_fields
            
            total = sum(data[seculo].values())
            pos = 1
            ref_value = None
            
            i = 0
            for nome in data[seculo]:
                freq = data[seculo][nome]
                
                if ref_value is None:
                    ref_value = freq

                elif freq < ref_value:
                    pos = i+1;
                    ref_value = freq
            
                if pos <= 5:
                    percentage = freq / total * 100
                    
                    precision = 2
                    while round(percentage, precision) == 0.00:
                        precision += 1
                        
                    table.add_row([pos, nome, freq, '{:.{}f} %'.format(percentage, precision)])
                    
                else:
                    break
                
                i+=1
                
            print(table)
            print("\n", end='')

### Programas implementados

> a) Calcula a frequência de processos por ano (primeiro elemento da data);

> Obs: ***Ficheiro 'TPC3_ExA.py'***

In [ ]:
def processos_ano():
    file = open("processos.txt", "r")
    proc_ano = dict()
    for line in file:
        r = re.match(r"\d+::(?P<ano>\d+)-\d{2}-\d{2}::([A-Z][a-z]*) ([A-Z][a-z]* ){0,}([A-Z][a-z]*)", line)
        if r is not None:
            ano = int(r.group('ano'))
            if ano in proc_ano:
                proc_ano[ano] += 1
            else:
                proc_ano[ano] = 1
                
    file.close()
    return dict(sorted(proc_ano.items(), key=lambda x: x[1], reverse=True))

def main():
    proc_ano = processos_ano()
    print_distribuicao("Processos por ano", ["Ano", "Nº de Processos", "Percentagem (%)"], proc_ano)

if __name__ == '__main__':
    main()

> b) Calcula a frequência de nomes próprios (o primeiro em cada nome) e apelidos (o ultimo em cada nome) por séculos e apresenta os 5 mais usados;

> Obs: ***Ficheiro 'TPC3_ExB.py'***

In [ ]:
def nomes_e_apelidos():
    file = open("processos.txt", "r")
    dict_pnomes = dict()
    dict_apelidos = dict()
    
    for line in file:
        m_ano = re.match(r"\d+::(?P<ano>\d+)-\d{2}-\d{2}::([A-Z][a-z]*) ([A-Z][a-z]* ){0,}([A-Z][a-z]*)", line)
        fall_nomes = re.findall(r"::([A-Z][a-z]*) ([A-Z][a-z]* ){0,}([A-Z][a-z]*)", line)
        
        if m_ano is not None and fall_nomes is not None:
            ano = int(m_ano.group('ano'))
            seculo = (ano - 1) // 100 + 1

            fall_nomes = fall_nomes[:3]
            for match in fall_nomes:
                primeiro_nome = match[0]
                apelido = match[-1]
                
                if seculo in dict_pnomes:
                    if primeiro_nome in dict_pnomes[seculo]:
                        dict_pnomes[seculo][primeiro_nome] += 1
                    else:
                        dict_pnomes[seculo][primeiro_nome] = 1
                else:
                    dict_pnomes[seculo] = dict()
                    dict_pnomes[seculo][primeiro_nome] = 1
                    
                if seculo in dict_apelidos:
                    if apelido in dict_apelidos[seculo]:
                        dict_apelidos[seculo][apelido] += 1
                    else:
                        dict_apelidos[seculo][apelido] = 1
                else:
                    dict_apelidos[seculo] = dict()
                    dict_apelidos[seculo][apelido] = 1     
    
    file.close()
    
    dict_pnomes = dict(sorted(dict_pnomes.items(), reverse=True))
    for seculo in dict_pnomes:
        dict_pnomes[seculo] = dict(sorted(dict_pnomes[seculo].items(), key=lambda x: x[1], reverse=True))
    
    dict_apelidos = dict(sorted(dict_apelidos.items(), reverse=True))
    for seculo in dict_apelidos:
        dict_apelidos[seculo] = dict(sorted(dict_apelidos[seculo].items(), key=lambda x: x[1], reverse=True))

    return (dict_pnomes, dict_apelidos)

dict_pnomes, dict_apelidos = nomes_e_apelidos()
print_top5_nomes_por_seculo("Top 5 de primeiros nomes no século", ["Pos.", "Primeiro nome", "Frequência", "Percentagem (%)"], dict_pnomes)
print("\n")
print_top5_nomes_por_seculo("Top 5 de apelidos no século", ["Pos.", "Apelido", "Frequência", "Percentagem (%)"], dict_apelidos)

> c) Calcula a frequência dos vários tipos de relação: irmão, sobrinho, etc.;

> Obs: ***Ficheiro 'TPC3_ExC.py'***

In [ ]:
def relacoes():
    file = open("processos.txt", "r")
    dict_relacoes = dict()

    for line in file:
        r = re.match(r"\d+::(?P<ano>\d+)-\d{2}-\d{2}::([A-Z][a-z]*) ([A-Z][a-z]* ){0,}([A-Z][a-z]*)", line)
        fall_relacoes = re.findall(r",([a-zA-z]+)((?: +[a-zA-z]+)*)\. +Proc.\d+\.", line)
        
        if r is not None:  
            for match in fall_relacoes:
                rel = ''
                for p in match:
                    rel += p
                
                if rel in dict_relacoes:
                    dict_relacoes[rel] += 1
                else:
                    dict_relacoes[rel] = 1
                        
    file.close()
    return dict(sorted(dict_relacoes.items(), key=lambda x: x[1], reverse=True))


dict_relacoes = relacoes()
print_distribuicao("Relações presentes em registos de processos", ["Relação", "Frequência", "Percentagem (%)"], dict_relacoes)

> d) Converta os 20 primeiros registos num novo ficheiro de output mas em formato **Json**.

> Obs: ***Ficheiro 'TPC3_ExD.py'***

In [ ]:
def first_20_registers_to_json():
    first_20_lines = []
    nome_campos = ['id','data','nome','pai','mãe','observações']
    
    read_file = open("processos.txt", "r")
    for line in read_file:
        r = re.match(r"\d+::(?P<ano>\d+)-\d{2}-\d{2}::([A-Z][a-z]*) ([A-Z][a-z]* ){0,}([A-Z][a-z]*)", line)
        if r is not None:
            campos = re.split("::", line)
            line_to_json = dict()
            for i in range(6):
                line_to_json[nome_campos[i]] = campos[i]
            first_20_lines.append(line_to_json)
        
        if len(first_20_lines) >= 20:
            break
    
    read_file.close()
        
    dict_processos = dict()
    dict_processos['processos'] = first_20_lines
    
    write_file = open("processos20.json", "w")
    json.dump(dict_processos, write_file, ensure_ascii=False, indent=2)    
    write_file.close()

            
first_20_registers_to_json()